CSI 4106 - Introduction to Artificial Intelligence - Project W2022

Simon Paquette - spaqu044@uottawa.ca - 300044038

Image Colorization


In [ ]:
from collections import Counter, OrderedDict
from pathlib import Path

import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.layers import (
    Conv2D,
    Input,
    LeakyReLU,
    ReLU,
    RepeatVector,
    Reshape,
    concatenate,
    UpSampling2D,
    BatchNormalization,
    MaxPooling2D,
)
from keras.models import Model, load_model
from sklearn.model_selection import train_test_split
%matplotlib inline


In [ ]:
# my gpu: GTX1650 4gb
import GPUtil

GPUs = GPUtil.getGPUs()
for i, gpu in enumerate(GPUs):
    print(
        "GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%".format(
            i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil * 100
        )
    )


# IDEA

- LAB vs LUV
- normalize data before
- split train/validation/test
- tensorboard
- callback
- checkpoint
- conv2d/relu/maxpool2D
- random flip
- k fold
- validation error vs train error
- leaky + tanh


In [ ]:
IMAGES_DIR = Path("images")
MODELS_DIR = Path("models")

WIDTH = 128
HEIGHT = 128
SIZE = (WIDTH, HEIGHT)


In [ ]:
def open_image(image_path: Path) -> np.ndarray:
    """
    Open and resized an image from the pathname

    Args:
        image_path (Path): pathname of the image

    Returns:
        np.ndarray: an opened image
    """
    opened_image = cv.imread(str(image_path), cv.IMREAD_COLOR)
    resized_image = cv.resize(opened_image, SIZE)
    image = resized_image.astype("uint8")
    return image


def plot_images(image_dict: dict):
    """
    Show the given images (1-4) side by side. 

    Args:
        image_dict (dict): key: title, value: image
    """
    keys = list(image_dict.keys())
    n = len(image_dict)
    assert n <= 4, "Limit of 4 images side by side"
    fig, images = plt.subplots(1, n)
    fig.set_size_inches(n * 5, 5)
    if n == 1:
        images = [images]
    for index, im in enumerate(images):
        im.axis("off")
        title = keys[index]
        im.set_title(title)
        image = image_dict[title]
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        im.imshow(image)


def bgr_to_l_ab_channels(bgr_image: np.ndarray) -> "tuple[np.ndarray, np.ndarray]":
    """
    From a bgr image, convert to the l* and a*b* channels

    Args:
        bgr_image (np.ndarray): an opened bgr image

    Returns:
        tuple[np.ndarray, np.ndarray]: the l* channel and the ab* channel
    """
    lab_image = cv.cvtColor(bgr_image, cv.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv.split(lab_image)
    ab_channel = cv.merge([a_channel, b_channel])
    return (l_channel, ab_channel)


def l_ab_channels_to_bgr(l_channel: np.ndarray, ab_channel: np.ndarray) -> np.ndarray:
    """
    Create a bgr image from merging a l* channel and a a*b* channel

    Args:
        l_channel (np.ndarray): l* channel
        ab_channel (np.ndarray): a*b* channel

    Returns:
        np.ndarray: an opened bgr image
    """
    merged_image = cv.merge([l_channel, ab_channel])
    bgr_image = cv.cvtColor(merged_image, cv.COLOR_LAB2BGR)
    return bgr_image


def counter_ndarray(array: np.ndarray) -> OrderedDict:
    """
    Helper function:
    Apply a counter to an array to evaluate the distribution of all pixel values

    Args:
        array (np.ndarray): an numpy array

    Returns:
        OrderedDict: the dict of a counter
    """
    flat = array.flatten()
    counts = dict(Counter(flat))
    ordered = OrderedDict(sorted(counts.items()))
    return ordered


def get_train_test_from_dir(images_dir: Path) -> tuple:
    """
    Get data and labels for training and testing

    Args:
        images_dir (Path): pathname directory containing all the images

    Returns:
        tuple: x_train, x_test, y_train, y_test
    """
    pathnames = list(images_dir.iterdir())
    x_image, y_image = [], []
    for pathname in pathnames:
        image = open_image(pathname)
        l_channel, ab_channel = bgr_to_l_ab_channels(image)
        x_image.append(l_channel)
        y_image.append(ab_channel)
    data = np.array(x_image)
    labels = np.array(y_image)
    x_train, x_test, y_train, y_test = train_test_split(
        data, labels, test_size=0.2, shuffle=True
    )
    return (x_train, x_test, y_train, y_test)


def predict_image(array: np.ndarray, model, batch_size: int = None) -> np.ndarray:
    """
    Color prediction from an image (using the grayscale) created by the model

    Args:
        array (np.ndarray): an opened image
        model (_type_): a TF model for colorization
        batch_size (int, optional): bacth size to predict. Defaults to None.

    Returns:
        np.ndarray: a new predicted colored image
    """
    l_channel, ab_channel = bgr_to_l_ab_channels(array)
    grayscale = l_channel.reshape(1, HEIGHT, WIDTH, 1)
    ab_prediction = model.predict(grayscale, batch_size=batch_size)
    ab_prediction = ab_prediction.reshape(HEIGHT, WIDTH, 2)
    ab_prediction[ab_prediction < 0] = 0
    ab_prediction = ab_prediction.astype("uint8")
    new_image = l_ab_channels_to_bgr(l_channel, ab_prediction)
    return new_image


def create_model_colorization(model_func):
    """
    Define the colorization model

    Args:
        model_func (_type_): a function that create a TF model with different testing layer

    Returns:
        _type_: a TF colorization model with layers
    """
    input_layer = Input(shape=(HEIGHT, WIDTH, 1))
    output_layer = model_func(input_layer)
    model_colorization = Model(inputs=input_layer, outputs=output_layer)
    model_colorization.summary()
    return model_colorization


def train_pipeline(
    model_func, compile_func, fit_func, data: np.ndarray, labels: np.ndarray
):
    """
    A pipeline to apply a training step. A model creation, compilation and fit from training data and labels

    Args:
        model_func (_type_): a function that create a TF model with different testing layers
        compile_func (_type_): a function that compile a TF model with different testing optimizers, losses and metrics
        fit_func (_type_): a function that fit a TF model with different testing parameters like epochs
        data (np.ndarray): training data
        labels (np.ndarray): training labels

    Returns:
        _type_: a trained TF colorization model
    """
    model = create_model_colorization(model_func)
    compile_func(model)
    fit_func(model, data, labels)
    return model


def test_eval(model, data: np.ndarray, labels: np.ndarray, batch_size: int = None):
    """
    Evaluate the model with testing data and labels

    Args:
        model (_type_): a trained TF colorization model
        data (np.ndarray): testing data
        labels (np.ndarray): testing labels
        batch_size (int, optional): bacth size to evaluate. Defaults to None.
    """
    model.evaluate(data, labels, batch_size=batch_size)


def save_my_model(model, name: str):
    """
    Save a model into the folder MODELS_DIR

    Args:
        model (_type_): a trained model
        name (str): model name
    """
    model.save(f"{MODELS_DIR}/{name}.h5")


def load_my_model(name: str):
    """
    Load a model from the folder MODELS_DIR

    Args:
        name (str): model name

    Returns:
        _type_: a trained model
    """
    model = load_model(f"{MODELS_DIR}/{name}.h5")
    return model


In [ ]:
def model_basic_conv(input_layer):
    model = Conv2D(16, (3, 3), activation="relu", padding="same", strides=1)(
        input_layer
    )
    model = Conv2D(32, (3, 3), activation="relu", padding="same", strides=1)(model)
    model = Conv2D(64, (3, 3), activation="relu", padding="same", strides=1)(model)
    model = Conv2D(32, (3, 3), activation="relu", padding="same", strides=1)(model)
    model = Conv2D(16, (3, 3), activation="relu", padding="same", strides=1)(model)
    model = Conv2D(2, (3, 3), activation="relu", padding="same", strides=1)(model)
    return model


def model_conv_sampling(input_layer):
    model = Conv2D(64, (3, 3), activation="relu", padding="same")(input_layer)
    model = Conv2D(64, (3, 3), activation="relu", padding="same", strides=2)(model)
    model = Conv2D(128, (3, 3), activation="relu", padding="same")(model)
    model = Conv2D(128, (3, 3), activation="relu", padding="same", strides=2)(model)
    model = Conv2D(256, (3, 3), activation="relu", padding="same")(model)
    model = Conv2D(256, (3, 3), activation="relu", padding="same", strides=2)(model)
    model = Conv2D(512, (3, 3), activation="relu", padding="same")(model)
    model = Conv2D(256, (3, 3), activation="relu", padding="same")(model)
    model = Conv2D(128, (3, 3), activation="relu", padding="same")(model)
    model = UpSampling2D((2, 2))(model)
    model = Conv2D(64, (3, 3), activation="relu", padding="same")(model)
    model = UpSampling2D((2, 2))(model)
    model = Conv2D(32, (3, 3), activation="relu", padding="same")(model)
    model = Conv2D(2, (3, 3), activation="relu", padding="same")(model)
    model = UpSampling2D((2, 2))(model)
    return model


def model_rev_conv(input_layer):
    model = Conv2D(128, (3, 3), activation="relu", padding="same", strides=1)(
        input_layer
    )
    model = Conv2D(64, (3, 3), activation="relu", padding="same", strides=1)(model)
    model = Conv2D(32, (3, 3), activation="relu", padding="same", strides=1)(model)
    model = Conv2D(16, (3, 3), activation="relu", padding="same", strides=1)(model)
    model = Conv2D(32, (3, 3), activation="relu", padding="same", strides=1)(model)
    model = Conv2D(64, (3, 3), activation="relu", padding="same", strides=1)(model)
    model = Conv2D(128, (3, 3), activation="relu", padding="same", strides=1)(model)
    model = Conv2D(2, (3, 3), activation="relu", padding="same", strides=1)(model)
    return model


def compile_adam(model):
    model.compile(optimizer="adam", loss="mse")


def compile_rmsprop(model):
    model.compile(optimizer="rmsprop", loss="mse")


def fit_2(model, data, labels):
    model.fit(data, labels, epochs=2, batch_size=8, shuffle=True)


def fit_10(model, data, labels):
    model.fit(data, labels, epochs=10, batch_size=8, shuffle=True)


def fit_25(model, data, labels):
    model.fit(data, labels, epochs=25, batch_size=8, shuffle=True)


def fit_50(model, data, labels):
    model.fit(data, labels, epochs=50, batch_size=8, shuffle=True)


In [ ]:
x_train, x_test, y_train, y_test = get_train_test_from_dir(IMAGES_DIR)


In [ ]:
model_name = "rev_conv_adam_10"
model_layers = model_rev_conv
model_compile = compile_adam
model_fit = fit_10

print("TRAIN")
model = train_pipeline(model_layers, model_compile, model_fit, x_train, y_train)
save_my_model(model, model_name)
print("\n\nTEST\n----")
test_eval(model, x_test, y_test)


In [ ]:
n_jpg = 6000
image_path = Path(IMAGES_DIR, f"{n_jpg}.jpg")

name_1 = "basic_conv_adam_2"
name_2 = "rev_conv_adam_10"

model_1 = load_my_model(name_1)
model_2 = load_my_model(name_2)
raw_image = open_image(image_path)
grayscale, _ = bgr_to_l_ab_channels(raw_image)
new_image_1 = predict_image(raw_image, model_1)
new_image_2 = predict_image(raw_image, model_2)

im_show = {
    "REAL IMAGE": raw_image,
    "GRAYSCALE": grayscale,
    "PREDICT 1": new_image_1,
    "PREDICT 2": new_image_2,
}
plot_images(im_show)
